In [ ]:
#!pip install --upgrade google-cloud-speech
#-----------------------------------------------
!pip install pyspellchecker
!git clone https://github.com/barrust/pyspellchecker.git
!cd pyspellchecker
!python -m build
#-----------------------------------------------
! pip install -q tensorflow-text
! pip install tensorflow_addons
! pip install transformers
#-----------------------------------------------
!pip install lazypredict
#-----------------------------------------------
!nltk.download('stopwords')
#-----------------------------------------------
!pip install translate
#-----------------------------------------------
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 18.6 MB/s eta 0:00:00
Cloning into 'pyspellchecker'...
remote: Enumerating objects: 712, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 712 (delta 33), reused 61 (delta 24), pack-reused 633
Receiving objects: 100% (712/712), 68.53 MiB | 19.58 MiB/s, done.
Resolving deltas: 100% (385/385), done.
/usr/bin/python3: No module named build
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 KB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 25.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simp

In [ ]:
import os
#from google.cloud import speech
import numpy as np
import pandas as pd
import string
import re
#-----------------------------------------------
from spellchecker import SpellChecker
#-----------------------------------------------
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow_addons as tfa
import keras
from keras import backend as k
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

#-----------------------------------------------
from sklearn.model_selection import train_test_split
#-----------------------------------------------
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.svm import NuSVC
from sklearn import svm
import lightgbm as lgb
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import f1_score
#-----------------------------------------------
from lazypredict.Supervised import LazyClassifier
from translate import Translator
#-----------------------------------------------
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
#-----------------------------------------------
import fasttext

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


# Choose Question

In [ ]:
question=1

# Data Pre-Processing functions





In [ ]:
num2words = {1: 'One', 2: 'Two', 3: 'Three', 4: 'Four', 5: 'Five', \
             6: 'Six', 7: 'Seven', 8: 'Eight', 9: 'Nine', 10: 'Ten', \
            11: 'Eleven', 12: 'Twelve', 13: 'Thirteen', 14: 'Fourteen', \
            15: 'Fifteen', 16: 'Sixteen', 17: 'Seventeen', 18: 'Eighteen', \
            19: 'Nineteen', 20: 'Twenty', 30: 'Thirty', 40: 'Forty', \
            50: 'Fifty', 60: 'Sixty', 70: 'Seventy', 80: 'Eighty', \
            90: 'Ninety', 0: 'Zero'}

def n2w(n):
  try:
      return num2words[n]
  except KeyError:
      try:
          return (num2words[n-n%10] +' '+ num2words[n%10].lower())
      except KeyError:
          return '-99'

def remove_extra_space(text):
    return re.sub(' +', ' ', text)

punctuation = string.punctuation + "؟'«»٫٬،.؛!()-[]{};:\,<>./?@$%^&*_~"
def remove_punctuation(text):
    #punctuation = string.punctuation + "؟'«»٫٬،.؛"
    return text.translate(str.maketrans('', '', punctuation))

translator= Translator(to_lang="Arabic")
def change_numbers(text):
  try:
    text_original=text
    text=text.split()
    for i in range (len(text)):
      if(text[i].isnumeric()):
        english=n2w(int(text[i]))
        translation = translator.translate(english)
        text[i]=translation
    text=" ".join(text)
    return text
  except:
    return text_original

spell = SpellChecker(language='ar')   # use the Arabic Dictionary
def separate_waw(text):
  splitted_text=text.split()
  for j in range(0,len(splitted_text)): 
      if((splitted_text[j][0]=='و') and (splitted_text[j]!=spell.correction(splitted_text[j]))):
              splitted_text[j] = splitted_text[j][:0+1] + ' ' + splitted_text[j][0+1:]
  text=" ".join(splitted_text)
  return text

# Reference Dataset

In [ ]:
ref_df=pd.read_csv('/content/drive/MyDrive/Speech Dataset/sas-ar.csv')
ref_df=ref_df[ref_df['EssaySet']==question]
ref_df.reset_index(inplace=True)

In [ ]:
#Reference Dataset Preprocessing

ref_df["EssayText"] = ref_df["EssayText"].apply(lambda text: remove_punctuation(text))
ref_df["EssayText"] = ref_df["EssayText"].apply(lambda text: remove_extra_space(text))
#ref_df["EssayText"] = ref_df["EssayText"].apply(lambda text: change_numbers(text))
ref_df["EssayText"] = ref_df["EssayText"].apply(lambda text: separate_waw(text))
refs=ref_df['EssayText']
refs=np.array(refs)

#Exchaning similar letters 

for i in range(0,len(refs)):
    ref=refs[i]
    #ref = ref.translate({ord(''): ''}) 
    ref = ref.translate({ord('ة'): 'ه'})
    ref = ref.translate({ord('ً'): None})
    ref = ref.translate({ord('أ'): 'ا'}) 
    ref = ref.translate({ord('إ'): 'ا'}) 
    refs[i]=ref

ref_df["EssayText"]=refs
    

#Exchaning letters with a manual approach 
    
# for i in range(0,len(refs)):
#     ref=refs[i]
#     ref=list(ref)
#     for j in range(0,len(ref)):
#         if(ref[j]=='ة'):
#             ref[j]='ه'
#         if(ref[j]==' ً'):
#             ref=ref[0 : j : ] + ref[j + 1 : :]
#         ref_joined="".join(ref)
#     refs[i]=ref_joined

# Converted Dataset

In [ ]:
#Dropping nulls from df and ref_df
null_df=df.isnull()
nulls_index=null_df[null_df['EssayText']==True].index
for x in nulls_index:
  #ref_df.drop(x,axis=0, inplace=True)

df.dropna(inplace=True)

df.reset_index(inplace=True)
ref_df.reset_index(inplace=True)

In [ ]:
#Dataset Preprocessing

df["EssayText"] = df["EssayText"].apply(lambda text: separate_waw(text))
df["EssayText"] = df["EssayText"].apply(lambda text: remove_punctuation(text))
df["EssayText"] = df["EssayText"].apply(lambda text: remove_extra_space(text))

# Conversion Accuracy "Word Error Rate"

In [ ]:
def wer(ref, hyp ,debug=True):
    r = ref.split()
    h = hyp.split()
    #costs will holds the costs, like in the Levenshtein distance algorithm
    costs = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]
    # backtrace will hold the operations we've done.
    # so we could later backtrace, like the WER algorithm requires us to.
    backtrace = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]
 
    OP_OK = 0
    OP_SUB = 1
    OP_INS = 2
    OP_DEL = 3
    DEL_PENALTY = 1
    INS_PENALTY = 1
    SUB_PENALTY = 1
    
    # First column represents the case where we achieve zero
    # hypothesis words by deleting all reference words.
    for i in range(1, len(r)+1):
        costs[i][0] = DEL_PENALTY*i
        backtrace[i][0] = OP_DEL
    
    # First row represents the case where we achieve the hypothesis
    # by inserting all hypothesis words into a zero-length reference.
    for j in range(1, len(h) + 1):
        costs[0][j] = INS_PENALTY * j
        backtrace[0][j] = OP_INS
    
    # computation
    for i in range(1, len(r)+1):
        for j in range(1, len(h)+1):
            if r[i-1] == h[j-1]:
                costs[i][j] = costs[i-1][j-1]
                backtrace[i][j] = OP_OK
            else:
                substitutionCost = costs[i-1][j-1] + SUB_PENALTY # penalty is always 1
                insertionCost    = costs[i][j-1] + INS_PENALTY   # penalty is always 1
                deletionCost     = costs[i-1][j] + DEL_PENALTY   # penalty is always 1
                 
                costs[i][j] = min(substitutionCost, insertionCost, deletionCost)
                if costs[i][j] == substitutionCost:
                    backtrace[i][j] = OP_SUB
                elif costs[i][j] == insertionCost:
                    backtrace[i][j] = OP_INS
                else:
                    backtrace[i][j] = OP_DEL
                 
    # back trace though the best route:
    i = len(r)
    j = len(h)
    numSub = 0
    numDel = 0
    numIns = 0
    numCor = 0
    if debug:
        print("OP\tREF\tHYP")
        lines = []
    while i > 0 or j > 0:
        if backtrace[i][j] == OP_OK:
            numCor += 1
            i-=1
            j-=1
            if debug:
                lines.append("OK\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_SUB:
            numSub +=1
            i-=1
            j-=1
            if debug:
                lines.append("SUB\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_INS:
            numIns += 1
            j-=1
            if debug:
                lines.append("INS\t" + "****" + "\t" + h[j])
        elif backtrace[i][j] == OP_DEL:
            numDel += 1
            i-=1
            if debug:
                lines.append("DEL\t" + r[i]+"\t"+"****")
    if debug:
        lines = reversed(lines)
        for line in lines:
            print(line)
        print("#cor " + str(numCor))
        print("#sub " + str(numSub))
        print("#del " + str(numDel))
        print("#ins " + str(numIns))
    # return (numSub + numDel + numIns) / (float) (len(r))
    wer_result = round( (numSub + numDel + numIns) / (float) (len(r)), 3)
    return {'WER':wer_result, 'numCor':numCor, 'numSub':numSub, 'numIns':numIns, 'numDel':numDel, "numCount": len(r)}
 
def wers(refs, hyps):
    numSub = 0
    numDel = 0
    numCor = 0
    numIns = 0
    numCount = 0
    for ref, hyp in zip(refs, hyps):
        result = wer(ref, hyp, False)
        numSub += result["numSub"]
        numDel += result["numDel"]
        numCor += result["numCor"]
        numIns += result["numIns"]
        numCount += result["numCount"]
    return round( (numSub + numDel + numIns) / (float) (numCount), 3)

In [ ]:
wers(ref_df['EssayText'],df['EssayText'])

# Embeddings/Lemma


In [ ]:
# !pip install farasapy
# from farasa.stemmer import FarasaStemmer
# !pip install -U sentence-transformers
# from sentence_transformers import SentenceTransformer
# import nltk
# from nltk.stem.isri import ISRIStemmer

In [ ]:
# !conda create -n env_pytorch python=3.6
# !conda activate env_pytorch
# !pip install torchvision 
# import torch
# import torchvision

In [ ]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")
def embed_text_use(df):
  X = np.array(embed(df['EssayText'])) 
  y = np.array(df["Score1"])
  return X,y

#model = SentenceTransformer('all-MiniLM-L6-v2')
def embed_text_BERT(df):
  embeddings = model.encode(df['EssayText'])
  X = np.array(embeddings) 
  y = np.array(df["Score1"])
  return X,y


#ft= fasttext.load_model('/content/drive/My Drive/Speech Dataset/cc.ar.300.bin')
def embed_fasttext2(sentence):
  return ft.get_sentence_vector(sentence)

def embed_fasttext(df):
  df["vector"] = df["EssayText"].apply(embed_fasttext2)
  X=[]
  for i in range(len(df["EssayText"])):
    X.append(df["vector"][i])
  X=np.array(X)
  y=np.array(df["Score1"])
  return X,y

#stop_words = set(stopwords.words('arabic'))
def removing_stop_words(text):
  word_tokens = word_tokenize(text)
  filtered_sentence = []  
  for w in word_tokens:
      if w not in stop_words:
          filtered_sentence.append(w)
  text=" ".join(filtered_sentence)
  return text

def lemmatization(df):
  #st = ISRIStemmer()
  fs = FarasaStemmer()
  corpus = []
  messages=df["EssayText"]
  for i in range(0, len(messages)):
      message = messages[i]
      stemmed_text = fs.stem(message)  
      corpus.append(stemmed_text)
  return corpus

In [ ]:
#Lemmatization

lemma=lemmatization(df)
df['EssayText']=lemma

In [ ]:
df["EssayText"] = df["EssayText"].apply(lambda text: change_numbers(text))
#df["EssayText"] = df["EssayText"].apply(lambda text: removing_stop_words(text))

X,y = embed_text_use(df)
#X,y = embed_text_BERT(df)
#X,y =embed_fasttext(df)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y ,test_size=0.2, random_state=42)
num_labels=len(np.unique(y_train))
print(np.unique(y_train, return_counts=True))

(array([0, 1, 2]), array([231, 614, 427]))


# Before Deep Learning

In [ ]:
X_train=np.expand_dims(X_train,axis=1)
X_test=np.expand_dims(X_test,axis=1)

In [ ]:
X_train.shape

(1272, 1, 512)

# LSTM

In [ ]:
tf.keras.backend.clear_session()
epochs = 7
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)
lstm_model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(1,X_train.shape[2])),
        tf.keras.layers.LSTM(128,activation="relu", return_sequences=True),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(64 ,activation="relu"),
        tf.keras.layers.Dense(num_labels, activation="softmax")
])
op = keras.optimizers.Adam(learning_rate=0.001)
loss = keras.losses.SparseCategoricalCrossentropy()
lstm_model.compile(optimizer=op,
              loss=loss,
              metrics=[tfa.metrics.CohenKappa(weightage="quadratic", sparse_labels=True, num_classes=num_labels),"accuracy"])
lstm_history = lstm_model.fit(X_train, y_train,epochs=epochs, validation_split=0.2, batch_size=8, callbacks=[callback], shuffle=True)

Epoch 1/7
128/128 [==============================] - 9s 31ms/step - loss: 0.9122 - cohen_kappa: 0.2900 - accuracy: 0.5919 - val_loss: 0.7013 - val_cohen_kappa: 0.5854 - val_accuracy: 0.7137
Epoch 2/7
128/128 [==============================] - 1s 9ms/step - loss: 0.7058 - cohen_kappa: 0.5279 - accuracy: 0.6991 - val_loss: 0.6570 - val_cohen_kappa: 0.6809 - val_accuracy: 0.7569
Epoch 3/7
128/128 [==============================] - 1s 9ms/step - loss: 0.6286 - cohen_kappa: 0.6248 - accuracy: 0.7434 - val_loss: 0.6265 - val_cohen_kappa: 0.7124 - val_accuracy: 0.7608
Epoch 4/7
128/128 [==============================] - 1s 9ms/step - loss: 0.5899 - cohen_kappa: 0.6567 - accuracy: 0.7532 - val_loss: 0.6064 - val_cohen_kappa: 0.7017 - val_accuracy: 0.7725
Epoch 5/7
128/128 [==============================] - 1s 9ms/step - loss: 0.5638 - cohen_kappa: 0.6825 - accuracy: 0.7679 - val_loss: 0.5945 - val_cohen_kappa: 0.7297 - val_accuracy: 0.7882
Epoch 6/7
128/128 [==============================] - 1

In [ ]:
print(lstm_model.evaluate(X_test,y_test))

10/10 [==============================] - 0s 8ms/step - loss: 0.7148 - cohen_kappa: 0.6248 - accuracy: 0.6865
[0.7147642970085144, 0.6247808933258057, 0.6865203976631165]


# RNN

In [ ]:
tf.keras.backend.clear_session()
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100)
epoch = 7
rnn_model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(1,X_train.shape[2])),
        tf.keras.layers.SimpleRNN(128,activation="relu", return_sequences=True),
        tf.keras.layers.Dropout(.2),
        tf.keras.layers.SimpleRNN(64 ,activation="relu"),
        #tf.keras.layers.Dropout(.2),
        tf.keras.layers.Dense(num_labels, activation="softmax"),
])
op = keras.optimizers.Adam(learning_rate=0.001)
loss = keras.losses.SparseCategoricalCrossentropy()
rnn_model.compile(optimizer=op,
              loss=loss,
              metrics=[tfa.metrics.CohenKappa(num_classes=num_labels, sparse_labels=True, weightage="quadratic"),"accuracy"])
rnn_history = rnn_model.fit(X_train, y_train,epochs=epoch, validation_split=0.1, batch_size=8, callbacks=[callback])

Epoch 1/7
143/143 [==============================] - 3s 9ms/step - loss: 0.8297 - cohen_kappa: 0.4460 - accuracy: 0.6477 - val_loss: 0.6337 - val_cohen_kappa: 0.7016 - val_accuracy: 0.7969
Epoch 2/7
143/143 [==============================] - 1s 5ms/step - loss: 0.6516 - cohen_kappa: 0.6412 - accuracy: 0.7325 - val_loss: 0.5850 - val_cohen_kappa: 0.6588 - val_accuracy: 0.7891
Epoch 3/7
143/143 [==============================] - 1s 5ms/step - loss: 0.5897 - cohen_kappa: 0.6856 - accuracy: 0.7692 - val_loss: 0.6528 - val_cohen_kappa: 0.6623 - val_accuracy: 0.7500
Epoch 4/7
143/143 [==============================] - 1s 5ms/step - loss: 0.5420 - cohen_kappa: 0.7100 - accuracy: 0.7876 - val_loss: 0.6432 - val_cohen_kappa: 0.6638 - val_accuracy: 0.7578
Epoch 5/7
143/143 [==============================] - 1s 5ms/step - loss: 0.5046 - cohen_kappa: 0.7399 - accuracy: 0.8059 - val_loss: 0.6306 - val_cohen_kappa: 0.6839 - val_accuracy: 0.7500
Epoch 6/7
143/143 [==============================] - 1s

In [ ]:
print(rnn_model.evaluate(X_test,y_test))

10/10 [==============================] - 1s 4ms/step - loss: 0.7266 - cohen_kappa: 0.6464 - accuracy: 0.7085
[0.7266185879707336, 0.6463850140571594, 0.7084639668464661]


# Bi-directional

In [ ]:
epochs = 5
tf.keras.backend.clear_session()
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)
bi_model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(1,X_train.shape[2])),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,activation="relu", return_sequences=True)),
        tf.keras.layers.Dropout(.2),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64 ,activation="relu")),
        #tf.keras.layers.Dropout(.2),
        tf.keras.layers.Dense(num_labels, activation="softmax"),
])
op = keras.optimizers.Adam(learning_rate=0.001)
loss = keras.losses.SparseCategoricalCrossentropy()
bi_model.compile(optimizer=op,
              loss=loss,
              metrics=[tfa.metrics.CohenKappa(weightage="quadratic", sparse_labels=True, num_classes=num_labels),"accuracy"])
bi_history = bi_model.fit(X_train, y_train,epochs=epochs, validation_split=0.1, batch_size=8, shuffle=True, callbacks=[callback])

Epoch 1/5
143/143 [==============================] - 17s 75ms/step - loss: 0.8612 - cohen_kappa: 0.3847 - accuracy: 0.6171 - val_loss: 0.6428 - val_cohen_kappa: 0.6099 - val_accuracy: 0.7344
Epoch 2/5
143/143 [==============================] - 9s 64ms/step - loss: 0.6647 - cohen_kappa: 0.6066 - accuracy: 0.7238 - val_loss: 0.6089 - val_cohen_kappa: 0.6856 - val_accuracy: 0.7891
Epoch 3/5
143/143 [==============================] - 9s 64ms/step - loss: 0.6019 - cohen_kappa: 0.6588 - accuracy: 0.7430 - val_loss: 0.5742 - val_cohen_kappa: 0.7329 - val_accuracy: 0.8125
Epoch 4/5
143/143 [==============================] - 10s 67ms/step - loss: 0.5688 - cohen_kappa: 0.7009 - accuracy: 0.7701 - val_loss: 0.6075 - val_cohen_kappa: 0.7316 - val_accuracy: 0.7812
Epoch 5/5
143/143 [==============================] - 9s 63ms/step - loss: 0.5372 - cohen_kappa: 0.7022 - accuracy: 0.7736 - val_loss: 0.5765 - val_cohen_kappa: 0.7351 - val_accuracy: 0.7891


In [ ]:
print(bi_model.evaluate(X_test,y_test))

10/10 [==============================] - 2s 86ms/step - loss: 0.6895 - cohen_kappa: 0.6643 - accuracy: 0.7085
[0.6895091533660889, 0.6642712354660034, 0.7084639668464661]


# Before Machine Learning

In [ ]:
X_train=X_train.reshape(X_train.shape[0],X_train.shape[2])
X_test=X_test.reshape(X_test.shape[0],X_test.shape[2])


# Lazy Predict

In [ ]:
clf=LazyClassifier(verbose=0,ignore_warnings=True,custom_metric=None)
models,predictions=clf.fit(X_train,X_test,y_train,y_test)
print(models)

100%|██████████| 29/29 [01:18<00:00,  2.70s/it]

                               Accuracy  Balanced Accuracy ROC AUC  F1 Score  \
Model                                                                          
NuSVC                              0.71               0.70    None      0.71   
SVC                                0.71               0.69    None      0.71   
LGBMClassifier                     0.69               0.69    None      0.69   
XGBClassifier                      0.70               0.69    None      0.70   
RandomForestClassifier             0.68               0.66    None      0.68   
ExtraTreesClassifier               0.67               0.65    None      0.67   
AdaBoostClassifier                 0.65               0.64    None      0.65   
NearestCentroid                    0.62               0.64    None      0.60   
SGDClassifier                      0.64               0.64    None      0.64   
RidgeClassifierCV                  0.64               0.63    None      0.63   
BernoulliNB                        0.60 

# SVM

In [ ]:
#clf = make_pipeline(StandardScaler(), SVC())
clf = svm.SVC(kernel='linear', C=5)
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
print("Accuracy:", clf.score(X_test, y_test))
print("QWK:", cohen_kappa_score(y_test, pred, weights='quadratic'))

print("F1 Score")
for i in range(num_labels):
  print("Label",i," F1 Score:", f1_score(y_test, pred,average=None)[i])


Accuracy: 0.6990595611285266
QWK: 0.6260420284821118
F1 Score
Label 0  F1 Score: 0.5208333333333334
Label 1  F1 Score: 0.7195121951219511
Label 2  F1 Score: 0.7476635514018691


# LogisticRegression 

In [ ]:
clf = LogisticRegression(random_state=0)
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
print("Accuracy:", clf.score(X_test, y_test))
print("QWK:", cohen_kappa_score(y_test, pred, weights='quadratic'))

print("F1 Score")
for i in range(num_labels):
  print("Label",i," F1 Score:", f1_score(y_test, pred,average=None)[i])

Accuracy: 0.7115987460815048
QWK: 0.6453706014474669
F1 Score
Label 0  F1 Score: 0.5057471264367817
Label 1  F1 Score: 0.7337278106508877
Label 2  F1 Score: 0.76056338028169



# KNeighborsClassifier

In [ ]:
clf = KNeighborsClassifier(n_neighbors=9)
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
print("Accuracy:", clf.score(X_test, y_test))
print("QWK:", cohen_kappa_score(y_test, pred, weights='quadratic'))

print("F1 Score")
for i in range(num_labels):
  print("Label",i," F1 Score:", f1_score(y_test, pred,average=None)[i])

Accuracy: 0.5574229691876751
QWK: 0.34214769429216385
F1 Score
Label 0  F1 Score: 0.689655172413793
Label 1  F1 Score: 0.2876712328767123
Label 2  F1 Score: 0.4691358024691358


# ExtraTreesClassifier

In [ ]:
clf = ExtraTreesClassifier(n_estimators=50, random_state=0)
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
print("Accuracy:", clf.score(X_test, y_test))
print("QWK:", cohen_kappa_score(y_test, pred, weights='quadratic'))

print("F1 Score")
for i in range(num_labels):
  print("Label",i," F1 Score:", f1_score(y_test, pred,average=None)[i])

Accuracy: 0.5826330532212886
QWK: 0.21158068057080126
F1 Score
Label 0  F1 Score: 0.717434869739479
Label 1  F1 Score: 0.24561403508771934
Label 2  F1 Score: 0.297029702970297


# RandomForestClassifier

In [ ]:
clf = RandomForestClassifier(n_estimators=10,max_depth=9,random_state=0)
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
print("Accuracy:", clf.score(X_test, y_test))
print("QWK:", cohen_kappa_score(y_test, pred, weights='quadratic'))

print("F1 Score")
for i in range(num_labels):
  print("Label",i," F1 Score:", f1_score(y_test, pred,average=None)[i])

Accuracy: 0.5518207282913166
QWK: 0.19820959731755927
F1 Score
Label 0  F1 Score: 0.7033898305084745
Label 1  F1 Score: 0.24
Label 2  F1 Score: 0.27350427350427353


# LGBMClassifier

In [ ]:

clf = lgb.LGBMClassifier()
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
print("Accuracy:", clf.score(X_test, y_test))
print("QWK:", cohen_kappa_score(y_test, pred, weights='quadratic'))

print("F1 Score")
for i in range(num_labels):
  print("Label",i," F1 Score:", f1_score(y_test, pred,average=None)[i])

Accuracy: 0.6833855799373041
QWK: 0.6263627353815658
F1 Score
Label 0  F1 Score: 0.47727272727272735
Label 1  F1 Score: 0.7023809523809524
Label 2  F1 Score: 0.7383177570093457


# XGBClassifier

In [ ]:
clf = XGBClassifier(learning_rate =0.05,
 n_estimators=50,
 max_depth=3,
 #min_child_weight=1,
 #gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'multi:softprob',
 nthread=4,
 seed=27)

#xgbc = XGBClassifier()
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
print("Accuracy:", clf.score(X_test, y_test))
print("QWK:", cohen_kappa_score(y_test, pred, weights='quadratic'))

print("F1 Score")
for i in range(num_labels):
  print("Label",i," F1 Score:", f1_score(y_test, pred,average=None)[i])

Accuracy: 0.5658263305322129
QWK: 0.24501025268530363
F1 Score
Label 0  F1 Score: 0.7112970711297072
Label 1  F1 Score: 0.22399999999999998
Label 2  F1 Score: 0.32432432432432434



# BaggingClassifier

In [ ]:
clf = BaggingClassifier(n_estimators=150,random_state=0)
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
print("Accuracy:", clf.score(X_test, y_test))
print("QWK:", cohen_kappa_score(y_test, pred, weights='quadratic'))

print("F1 Score")
for i in range(num_labels):
  print("Label",i," F1 Score:", f1_score(y_test, pred,average=None)[i])

Accuracy: 0.5994397759103641
QWK: 0.28854673081072857
F1 Score
Label 0  F1 Score: 0.7249999999999999
Label 1  F1 Score: 0.28333333333333327
Label 2  F1 Score: 0.4035087719298246


# NuSVC


In [ ]:
clf = make_pipeline(StandardScaler(), NuSVC())
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
print("Accuracy:", clf.score(X_test, y_test))
print("QWK:", cohen_kappa_score(y_test, pred, weights='quadratic'))

print("F1 Score")
for i in range(num_labels):
  print("Label",i," F1 Score:", f1_score(y_test, pred,average=None)[i])


Accuracy: 0.6358543417366946
QWK: 0.44950223899929675
F1 Score
Label 0  F1 Score: 0.7563805104408351
Label 1  F1 Score: 0.3918918918918919
Label 2  F1 Score: 0.5185185185185185


# LinearSVC

In [ ]:
clf = make_pipeline(StandardScaler(),LinearSVC(random_state=0, tol=1e-5))
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
print("Accuracy:", clf.score(X_test, y_test))
print("QWK:", cohen_kappa_score(y_test, pred, weights='quadratic'))

print("F1 Score")
for i in range(num_labels):
  print("Label",i," F1 Score:", f1_score(y_test, pred,average=None)[i])


Accuracy: 0.5126050420168067
QWK: 0.3203675848453511
F1 Score
Label 0  F1 Score: 0.6277777777777778
Label 1  F1 Score: 0.33333333333333337
Label 2  F1 Score: 0.4642857142857143


# SGDClassifier

In [ ]:
clf = make_pipeline(StandardScaler(),SGDClassifier(max_iter=1000, tol=1e-3))
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
print("Accuracy:", clf.score(X_test, y_test))
print("QWK:", cohen_kappa_score(y_test, pred, weights='quadratic'))

print("F1 Score")
for i in range(num_labels):
  print("Label",i," F1 Score:", f1_score(y_test, pred,average=None)[i])


Accuracy: 0.5602240896358543
QWK: 0.3764738588359008
F1 Score
Label 0  F1 Score: 0.6810810810810811
Label 1  F1 Score: 0.3957219251336898
Label 2  F1 Score: 0.4713375796178344




```
# This is formatted as code
```

# NearestCentroid

In [ ]:
clf = NearestCentroid()
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
print("Accuracy:", clf.score(X_test, y_test))
print("QWK:", cohen_kappa_score(y_test, pred, weights='quadratic'))

print("F1 Score")
for i in range(num_labels):
  print("Label",i," F1 Score:", f1_score(y_test, pred,average=None)[i])

Accuracy: 0.5378151260504201
QWK: 0.3163235285954561
F1 Score
Label 0  F1 Score: 0.6495726495726496
Label 1  F1 Score: 0.4
Label 2  F1 Score: 0.4574468085106383


# LinearDiscriminantAnalysis

In [ ]:
clf = LinearDiscriminantAnalysis()
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
print("Accuracy:", clf.score(X_test, y_test))
print("QWK:", cohen_kappa_score(y_test, pred, weights='quadratic'))

print("F1 Score")
for i in range(num_labels):
  print("Label",i," F1 Score:", f1_score(y_test, pred,average=None)[i])

Accuracy: 0.5098039215686274
QWK: 0.35184188582614306
F1 Score
Label 0  F1 Score: 0.6263736263736265
Label 1  F1 Score: 0.29189189189189185
Label 2  F1 Score: 0.496969696969697
